## Image distortions
### Daten: Gruppe von Erik, Fung, Isa
### *Seminar Visuelle Wahrnehmung beim Menschen und Bildqualität - SoSe 21* - G. Aguilar, M. Maertens


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from os import walk

# Pillow library, to manipulate images
from PIL import Image, ImageFilter


### Helper functions

In [1]:

# showing images
def show_2ims(im, im_b):
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(im, cmap='gray', vmin=0, vmax=255)

    plt.subplot(1,2,2)
    plt.imshow(im_b, cmap='gray', vmin=0, vmax=255)
    
    print('MSE', mse(im, im_b))
    
#show_2ims(im, im_b)

In [6]:
def show_2cuts(im1, im2):
    
    m = int(im.shape[0]/2) # middle row

    cut1 = im1[m, :]
    cut2 = im2[m, :]

    plt.figure()
    plt.plot(cut1, c='k', linewidth=1)
    plt.plot(cut2, c='b', linewidth=1)


### Manipulation d. impulse noise contamination

In [7]:
# we first make a copy of im

def noise(im_n, name, degree):
    
    im = np.array(im_n)
    im_d = np.copy(im)
    Nx = im.shape[0]
    Ny = im.shape[1]
    # number of pixels we want to manipulate
    #m = 38000
    m = int(Nx*Ny*degree)

    # take a random sample (without replacement) of size NxN out of number of pixels  
    spots_x = np.random.choice(range(Nx), m)
    spots_y = np.random.choice(range(Ny), m)

    # for each pixel we want to change
    for i in range(m):
        # we randomly assign either 0 (black) or 255 (white)
        im_d[spots_x[i], spots_y[i]] = np.random.randint(2)*255

    index = name.find('.')
    f = name[:index] + '_noise_' + str(degree) + name[index:]
    Image.fromarray(im_d).save('images/noise/'+ f, 'jpeg')
    return im_d

#im_n = noise(im, name,0.01)
#show_2ims(im, im_n)

#show_2cuts(im, im_d)


### Manipulation f. blurring

In [8]:
# we blur the image with gaussian filter. Implemented alread in scipy.
# sigma is the width of the gaussian filter
from scipy import ndimage

def blur(im, name, degree):

    global blu
    img = im.filter(ImageFilter.GaussianBlur(degree))
    im_b = np.array(img)
    index = name.find('.')
    f = name[:index] + '_blur_' + str(blu) + name[index:]
    Image.fromarray(im_b).save('images/blur/'+ f, 'jpeg')
    blu = blu + 1
    if blu > 5:
        blu = 1
    
    return im_b

#show_2ims(orig, blur(orig, name, 2))

#show_2cuts(im, im_f)


### Manipulation g. JPEG compression

In [9]:
# we save the image in very bad quality as JPG. JPEG Artifacts clearly appear. 
#Image.fromarray(im).convert('L').save('einstein_tmp.jpg', 'jpeg', quality=4)
def compress(im, name, degree, count):
    global com
    im_b = np.array(im)
    
    index = name.find('.')
    f = name[:index] + '_compression_' + str(count) + name[index:]
    Image.fromarray(im_b).save('images/compression/'+ f, 'jpeg', quality=degree)
    im_g = Image.open('images/compression/'+ f)#.convert('L')
    im_g = np.array(im_g)
    return im_g
    
# read the JPG image


#show_2ims(orig, compress(orig, name, 5))

#show_2cuts(im, im_g)

## Process all images

In [20]:
pathList = ['images/nice/', 'images/bad/']
noises = [0, 0.1,0.2,0.3,0.4, 0.5]
compressions = [1,5,7,14,40]
blurs = [4,3,2,1,0.2]

def process(paths):
    
    for path in paths:
        _, _, filenames = next(walk(path), (None, None, []))
        #filenames = list(map(lambda x: x.split('_')[0], filenames))
        #filenames.sort()
        for file in filenames:
            im_orig = Image.open(path + file)#.convert('L')

            # converting it to a numpy array. numpy arrays are easier to manipulate 
    
            #for n in noises:
                #noise(im_orig, file, n)
            for i in range(len(blurs)):
                blur(im_orig, file, blurs[i], i)
            for j in range(len(compressions)):
                compress(im_orig, file, compressions[j], j)
    print('done')
    #print(filenames)
    

process(pathList)

done


In [16]:
## END